# Implement Chroma DB
This might be an efficient way to stash documents and select them for the AI to analyze.

* https://docs.trychroma.com/getting-started

In [ ]:
%pip install chromadb
%pip freeze > requirements.txt

In [1]:
import chromadb
import json

chroma_client = chromadb.PersistentClient(path="llama-data/embeddings/chroma")

In [26]:
raw = open("./llama-data/data/apache_logs/4.txt").read().strip().split("\n")

#  https://www.geeksforgeeks.org/break-list-chunks-size-n-python/
chunk = 10
all_documents = [
    ((x * chunk) // chunk, raw[x * chunk : (x + 1) * chunk])
    for x in range((len(raw) + chunk - 1) // chunk)
]

for index, documents in all_documents:
    chroma_client.delete_collection(name=f"vec-{index}")
    collection = chroma_client.get_or_create_collection(name=f"vec-{index}")

    ids = range(len(documents), len(documents) * 2)
    collection.add(documents=documents, ids=[f"id-{x+1}" for x in ids])

In [28]:
results = []
anomaly_count = 2

for index, _ in all_documents:
    collection = chroma_client.get_collection(name=f"vec-{index}")
    document_count = len(collection.get()["ids"])
    iterations = round(document_count / anomaly_count)
    filter_out = []
    query = "*"

    for i in range(iterations):
        print(f"Index: {index+1}/{len(all_documents)}, Iteration: {i+1}/{iterations}")

        if len(filter_out) == 0:
            result = collection.query(
                query_texts=[query],
                n_results=anomaly_count,
            )
        
        else:
            exclusions = {"$and": [{"$not_contains": x} for x in filter_out]}
            result = collection.query(
                query_texts=[query],
                n_results=anomaly_count,
                where_document=exclusions,
            )
            
        query = result["documents"][0][0]
        [print(f"Filtering: {x[:15]}...") for x in result["documents"][0]]
        [filter_out.append(x) for x in result["documents"][0]]
        
        # This will be overwritten each round except at the end.
        results = result["documents"][0]


print(json.dumps(results, indent=4))

Index: 1/1, Iteration: 1/5
Filtering: 46.105.14.53 - ...
Filtering: 93.164.60.142 -...
Index: 1/1, Iteration: 2/5
Filtering: 86.182.52.145 -...
Filtering: 108.32.74.68 - ...
Index: 1/1, Iteration: 3/5
Filtering: 86.182.52.145 -...
Filtering: 86.182.52.145 -...
Index: 1/1, Iteration: 4/5
Filtering: 86.182.52.145 -...
Filtering: 86.182.52.145 -...
Index: 1/1, Iteration: 5/5
Filtering: 86.182.52.145 -...
Filtering: 108.32.74.68 - ...
[
    "86.182.52.145 - - [17/May/2015:12:05:10 +0000] \"GET /images/jordan-80.png HTTP/1.1\" 200 6146 \"http://www.semicomplete.com/articles/dynamic-dns-with-dhcp/\" \"Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1700.107 Safari/537.36\"",
    "108.32.74.68 - - [17/May/2015:12:05:42 +0000] \"GET /files/xdotool/docs HTTP/1.1\" 301 341 \"http://www.semicomplete.com/projects/xdotool/\" \"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/32.0.1700.102 Chrome/32.0.1700.102 Safari/537.36